In [2]:
import pandas as pd
from pathlib import Path

schooldata_load = Path("Resources/schools_complete.csv")
studentdata_load = Path("Resources/students_complete.csv")

school_data = pd.read_csv(schooldata_load)
student_data = pd.read_csv(studentdata_load)

schooldata_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
schooldata_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
school_count = len(schooldata_complete["school_name"].unique())
student_count = schooldata_complete["student_name"].count()
school_budget = schooldata_complete["budget"].sum()
avgmath_score = schooldata_complete["math_score"].mean()
avgread_score = schooldata_complete["reading_score"].mean()
passmath_percent = (schooldata_complete[schooldata_complete["math_score"]>=70].math_score.count()) / student_count
passread_percent = (schooldata_complete[schooldata_complete['reading_score'] >= 70].reading_score.count()) / student_count
allpass = (passmath_percent + passread_percent) / 2

In [4]:
dist_sum = pd.DataFrame({"Total Schools":[school_count],
                                 "Total Students":[student_count],
                                 "Total Budget":[school_budget],
                                 "Average Math Score":[avgmath_score],
                                 "Average Reading Score":[avgread_score],
                                 "% Passing Math":[passmath_percent*100],
                                 "% Passing Reading":[passread_percent*100],
                                 "% Overall Passing Rate":[allpass*100]})

dist_sum["Total Students"] = dist_sum["Total Students"].map("{:,}".format)
dist_sum["Total Budget"] = dist_sum["Total Budget"].map("${:,.2f}".format)

dist_sum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$82,932,329,558.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [5]:
school_summary = schooldata_complete.groupby(['school_name'])
school_names = schooldata_complete.school_name.sort_values().unique()
school_types = school_data.set_index(["school_name"])["type"]
school_studtotal = list(school_summary.student_name.count())
indv_budget = list(school_summary.budget.mean())
perstud_budget = [i/j for i,j in zip(indv_budget,school_studtotal)]
avg_math = list(school_summary.math_score.mean())
avg_reading = list(school_summary.reading_score.mean())
school_summary = schooldata_complete[schooldata_complete["math_score"]>=70].groupby(["school_name"])
school_passmathpct = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),school_studtotal)]
school_summary = schooldata_complete[schooldata_complete["reading_score"]>=70].groupby(["school_name"])
school_passreadpct = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),school_studtotal)]
schoolall_pass = [(i+j)/2 for i,j in zip(school_passmathpct,school_passreadpct)]

school_sum_df = pd.DataFrame({"School Names":school_names,
                             "School Type":school_types,
                             "Total Students":school_studtotal,
                             "Total School Budget":indv_budget,
                             "Per Student Budget":perstud_budget,
                             "Average Math Score":avg_math,
                             "Average Reading Score":avg_reading,
                             "% Passing Math":school_passmathpct,
                             "% Passing Reading":school_passreadpct,
                             "Overall Passing Rate":schoolall_pass})
school_sum_df = school_sum_df.reset_index(drop=True)
school_sum_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,District,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,Charter,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,Charter,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,Charter,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [6]:
top_5 = school_sum_df.sort_values(by="Overall Passing Rate", ascending=False).head(5).reset_index(drop=True)
top_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,District,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
1,Thomas High School,District,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
2,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
3,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
4,Wilson High School,District,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [7]:
bot_5 = school_sum_df.sort_values(by="Overall Passing Rate", ascending=True).head(5).reset_index(drop=True)
bot_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,Charter,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,Charter,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
3,Johnson High School,Charter,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
4,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [8]:
def avg_math_bygrade(grade):
    school_summary = schooldata_complete.loc[schooldata_complete.grade == grade].groupby(["school_name"])
    school_names = schooldata_complete.school_name.sort_values().unique()
    school_avgmath = list(school_summary.math_score.mean())
    
    avgmath_df = pd.DataFrame({"School Name":school_names,
                              f"{grade} Avg Math Score":school_avgmath})
    avgmath_df = avgmath_df.reset_index(drop=True)
    return avgmath_df

In [9]:
def avg_reading_bygrade(grade):
    school_summary = schooldata_complete.loc[schooldata_complete.grade == grade].groupby(["school_name"])
    school_names = schooldata_complete.school_name.sort_values().unique()
    school_avgread = list(school_summary.reading_score.mean())
    
    avgread_df = pd.DataFrame({"School Name":school_names,
                              f"{grade} Avg reading Score":school_avgread})
    avgread_df = avgread_df.reset_index(drop=True)
    return avgread_df

In [10]:
grade_9 = avg_math_bygrade('9th')
grade_10 = avg_math_bygrade('10th')
grade_11 = avg_math_bygrade('11th')
grade_12 = avg_math_bygrade('12th')
avg_math_score_by_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_math_score_by_grade = pd.merge(avg_math_score_by_grade,grade_11,how='inner',suffixes=('',''))
avg_math_score_by_grade = pd.merge(avg_math_score_by_grade,grade_12,how='inner',suffixes=('',''))
avg_math_score_by_grade

,School Name,9th Avg Math Score,10th Avg Math Score,11th Avg Math Score,12th Avg Math Score
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [11]:
grade_9 = avg_reading_bygrade('9th')
grade_10 = avg_reading_bygrade('10th')
grade_11 = avg_reading_bygrade('11th')
grade_12 = avg_reading_bygrade('12th')
avg_reading_score_by_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_reading_score_by_grade = pd.merge(avg_reading_score_by_grade,grade_11,how='inner',suffixes=('',''))
avg_reading_score_by_grade = pd.merge(avg_reading_score_by_grade,grade_12,how='inner',suffixes=('',''))
avg_reading_score_by_grade

,School Name,9th Avg reading Score,10th Avg reading Score,11th Avg reading Score,12th Avg reading Score
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [12]:
spend_bin = [0, 585, 615, 645, 675]
label = ["<$585", "$585-630", "$630-645", "$645-680"]

In [13]:
school_spend_df = school_sum_df.copy()

In [14]:
school_spend_df["Spending Ranges (Per Student)"] = pd.cut(school_sum_df["Per Student Budget"], spend_bin, labels=label)
school_spend_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$630-645
1,Cabrera High School,District,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
2,Figueroa High School,Charter,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$630-645
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$630-645
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$630-645
5,Hernandez High School,Charter,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-680
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-680
8,Johnson High School,Charter,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-680
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,$585-630


In [15]:
spend_math = school_spend_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spend_reading = school_spend_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spend_pass_math = school_spend_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spend_pass_reading = school_spend_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_pass_spend = school_spend_df.groupby(["Spending Ranges (Per Student)"])["Overall Passing Rate"].mean()

In [16]:
spend_sum = pd.DataFrame({"Average Math Score":spend_math,
                        "Average Reading Score":spend_reading,
                        "% Passing Math":spend_pass_math,
                        "% Passing Reading":spend_pass_reading,
                        "Overall Passing Rate":overall_pass_spend})
spend_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-630,83.599686,83.885211,94.230858,95.900287,95.065572
$630-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-680,76.997210,81.027843,66.164813,81.133951,73.649382


In [17]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_sum_df["School Size"] = pd.cut(school_sum_df["Total Students"], size_bins, labels=labels)
school_sum_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,Large (2000-5000)
1,Cabrera High School,District,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,Medium (1000-2000)
2,Figueroa High School,Charter,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,Large (2000-5000)
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,Large (2000-5000)
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,Medium (1000-2000)
5,Hernandez High School,Charter,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,Large (2000-5000)
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,Small (<1000)
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,Large (2000-5000)
8,Johnson High School,Charter,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,Large (2000-5000)
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,Small (<1000)


In [18]:
size_math_score = school_sum_df.groupby(["School Size"])["Average Math Score"].mean()
size_read_score = school_sum_df.groupby(["School Size"])["Average Reading Score"].mean()
size_pass_math = school_sum_df.groupby(["School Size"])["% Passing Math"].mean()
size_pass_read = school_sum_df.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_pass = school_sum_df.groupby(["School Size"])["Overall Passing Rate"].mean()

In [19]:
size_sum = pd.DataFrame({"Average Math Score":size_math_score,
                       "Average Reading Score":size_read_score,
                       "% Passing Math":size_pass_math,
                       "% Passing Reading":size_pass_read,
                       "Overall Passing Rate":size_overall_pass})
size_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [20]:
avg_math_type = school_sum_df.groupby(["School Type"])["Average Math Score"].mean()
avg_read_type = school_sum_df.groupby(["School Type"])["Average Reading Score"].mean()
avg_pct_pass_math_type = school_sum_df.groupby(["School Type"])["% Passing Math"].mean()
avg_pct_pass_reading_type = school_sum_df.groupby(["School Type"])["% Passing Reading"].mean()
avg_pct_overall_pass_type = school_sum_df.groupby(["School Type"])["Overall Passing Rate"].mean()

In [21]:
type_sum = pd.DataFrame({"Average Math Score":avg_math_type,
                       "Average Reading Score":avg_read_type,
                       "% Passing Math":avg_pct_pass_math_type,
                       "% Passing Reading":avg_pct_pass_reading_type,
                       "Overall Passing Rate":avg_pct_overall_pass_type})
type_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,80.324201,82.429369,79.873967,88.624209,84.249088
District,80.556334,82.643266,82.259154,89.898811,86.078983
